In [12]:
import sys
sys.path.append("../")

import pandas as pd
import os
import statsmodels.formula.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import print_function
from patsy import dmatrices
from openpyxl import load_workbook

from library import regulations
from library import characteristics
from library import analysis
from library import tables

In [14]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'cits.csv'),
                  sep=",", low_memory = False)
data.head()
data = data[data.year == 2016]
data['doi_16'] = np.where(data.doi_year == 2016, 1, 0)
data['doi_17'] = np.where(data.doi_year == 2017, 1, 0)
data['doi_18'] = np.where(data.doi_year == 2018, 1, 0)
data['tpsd'] = np.where(data.doi == False, 1, 0)
data.head()

,Unnamed: 0,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,...,treatyear,treatpost,treatpostyear,treatpostyear1,treatpostyear2,preyear,doi_16,doi_17,doi_18,tpsd
3365,4901,1902,CAYUGA ISD,2016,ANDERSON,N,M,Pass,True,H,...,-1.0,0,-0.0,0,0,-1.0,0,1,0,0
3366,4902,1903,ELKHART ISD,2016,ANDERSON,N,M,Pass,True,G,...,-2.0,0,-0.0,0,0,-2.0,0,0,1,0
3367,4903,1904,FRANKSTON ISD,2016,ANDERSON,N,M,Pass,True,H,...,-2.0,0,-0.0,0,0,-2.0,0,0,1,0
3368,4904,1906,NECHES ISD,2016,ANDERSON,N,M,Pass,True,H,...,-1.0,0,-0.0,0,0,-1.0,0,1,0,0
3369,4905,1907,PALESTINE ISD,2016,ANDERSON,N,M,Pass,True,E,...,0.0,1,0.0,1,0,0.0,1,0,0,0


# Characteristics by Adoption Year

In [16]:
y, X = dmatrices('students_num ~ doi + doi_16 + doi_18', data=data, return_type='dataframe', NA_action = "drop")
X.head()

,Intercept,doi[T.True],doi_16,doi_18
3365,1.0,1.0,0.0,0.0
3366,1.0,1.0,0.0,1.0
3367,1.0,1.0,0.0,1.0
3368,1.0,1.0,0.0,0.0
3369,1.0,1.0,1.0,0.0


In [17]:
mod = sm.OLS(y, X)    # Describe model
res = mod.fit()       # Fit model
print(res.summary())   # Summarize model

                            OLS Regression Results                            
Dep. Variable:           students_num   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.715
Date:                Sun, 17 Mar 2019   Prob (F-statistic):             0.0438
Time:                        14:09:05   Log-Likelihood:                -9286.5
No. Observations:                 842   AIC:                         1.858e+04
Df Residuals:                     838   BIC:                         1.860e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    6295.9137   1065.269      5.910      

# Make mini table

In [19]:
def coef_with_stars(coef, pvalue):
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(.1, .005)
test

'0.1**'

In [20]:
test = ['1', '2']

In [186]:
coef_1 = []
formula = 'math' + ' ~ ' + 'doi + doi_year'
y, X = dmatrices('students_num ~ doi + doi_year', data=data, return_type='dataframe', NA_action = "drop") 
mod = sm.OLS(y, X)    # Describe model
res = mod.fit()       # Fit model
print(res.summary())   # Summarize model
result = res.summary()
res.params['doi[T.True]']
res.bse['doi_year']
#coef_1.append(coef_with_stars(res.params['doi']))

                            OLS Regression Results                            
Dep. Variable:           students_num   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.342
Date:                Sat, 09 Mar 2019   Prob (F-statistic):              0.262
Time:                        13:11:37   Log-Likelihood:                -9346.7
No. Observations:                 848   AIC:                         1.870e+04
Df Residuals:                     845   BIC:                         1.871e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    5649.3484    998.064      5.660      

904.3584687994638

In [25]:
def make_coef_df(data, y_list, y_labels):
    regs = []
    cons = []
    coef_1 = []
    coef_2 = []
    coef_3 = []

    for var in y_list:
        formula = var + ' ~ ' + 'doi + doi_16 + doi_18'
        y, X = dmatrices(formula, data=data, return_type='dataframe', NA_action = "drop") 
        mod = sm.OLS(y, X)    # Describe model
        res = mod.fit()       # Fit model
        print(y_labels[var])
        print(res.summary())   # Summarize model
        cons.append(str(res.params["Intercept"].round(2)))
        cons.append('')
        coef_1.append(coef_with_stars(res.params['doi[T.True]'].round(2), res.pvalues['doi[T.True]']))
        coef_1.append('(' + str(res.bse['doi[T.True]'].round(2)) + ')')
        coef_2.append(coef_with_stars(res.params['doi_16'].round(2), res.pvalues['doi_16']))
        coef_2.append('(' + str(res.bse['doi_16'].round(2)) + ')')
        coef_3.append(coef_with_stars(res.params['doi_18'].round(2), res.pvalues['doi_18']))
        coef_3.append('(' + str(res.bse['doi_18'].round(2)) + ')')
        regs.append(y_labels[var])
        regs.append('')


    df = pd.DataFrame(
        {'Characteristic': regs,
         'Control': cons,
         'DOI Difference': coef_1,
         'Early Adopter': coef_2,
         'Late Adopter': coef_3,
         })
    return df
geo = make_coef_df(data = data, 
                                 y_list = characteristics.geography,
                                 y_labels = characteristics.labels)
geo

Urban
                            OLS Regression Results                            
Dep. Variable:             type_urban   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.206
Date:                Sun, 17 Mar 2019   Prob (F-statistic):             0.0226
Time:                        14:12:47   Log-Likelihood:                 49.091
No. Observations:                 842   AIC:                            -90.18
Df Residuals:                     838   BIC:                            -71.24
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0660      0.016      4.048

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Urban,0.07,-0.03,0.09**,0.04
1,,,(0.02),(0.03),(0.03)
2,Suburban,0.28,-0.04,0.09,-0.0
3,,,(0.04),(0.06),(0.05)
4,Town,0.23,0.05,-0.08,0.02
5,,,(0.04),(0.06),(0.05)
6,Rural,0.42,0.01,-0.11,-0.06
7,,,(0.04),(0.07),(0.06)


In [26]:
teacher = make_coef_df(data = data, 
                                 y_list = characteristics.teacher,
                                 y_labels = characteristics.labels)
teacher

Ave. Experience Teaching
                             OLS Regression Results                            
Dep. Variable:     teachers_tenure_ave   R-squared:                       0.001
Model:                             OLS   Adj. R-squared:                 -0.003
Method:                  Least Squares   F-statistic:                    0.1642
Date:                 Sun, 17 Mar 2019   Prob (F-statistic):              0.921
Time:                         14:13:02   Log-Likelihood:                -1715.4
No. Observations:                  842   AIC:                             3439.
Df Residuals:                      838   BIC:                             3458.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Ave. Experience Teaching,7.36,-0.1,0.03,-0.02
1,,,(0.16),(0.27),(0.21)
2,Teacher Turnover Ratio,0.19,-0.01,-0.01,0.0
3,,,(0.01),(0.01),(0.01)
4,Student-Teacher Ratio,6.49,0.2*,0.27,0.16
5,,,(0.1),(0.17),(0.13)


In [27]:
student = make_coef_df(data = data, 
                                 y_list = characteristics.student,
                                 y_labels = characteristics.labels)
student

Percent Hispanic
                            OLS Regression Results                            
Dep. Variable:          students_hisp   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     19.51
Date:                Sun, 17 Mar 2019   Prob (F-statistic):           3.12e-12
Time:                        14:13:21   Log-Likelihood:                -56.410
No. Observations:                 842   AIC:                             120.8
Df Residuals:                     838   BIC:                             139.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.5033      0.018

,Characteristic,Control,DOI Difference,Early Adopter,Late Adopter
0,Percent Hispanic,0.5,-0.16***,0.1**,0.03
1,,,(0.02),(0.04),(0.03)
2,Percent White,0.42,0.13***,-0.12**,-0.03
3,,,(0.02),(0.04),(0.03)
4,Percent Black,0.05,0.02*,0.0,0.0
5,,,(0.01),(0.01),(0.01)
6,Percent Econ. Disadvantaged,0.61,-0.07***,0.02,0.01
7,,,(0.02),(0.03),(0.02)
8,Average STAAR Performance (Std.),0.21,0.22***,0.04,-0.05
9,,,(0.06),(0.1),(0.08)


# To Table

In [28]:
rows = [6, 15, 22]

In [32]:
dfs = [geo, teacher, student]
file = table_path + 'regression_doi_year.xlsx'
wb = load_workbook(file)
ws = wb.active
str_n = 'N = ' + str(len(data[data.doi == False]))
ws.cell(row=4, column=2).value = str_n
str_n = 'N = ' + str(len(data[data.doi == True]))

for df, row in zip(dfs, rows):
    ws.cell(row=4, column=3).value = str_n
    row_n = row
    for ob in df.Control:
        ws.cell(row=row_n, column=2).value = ob
        row_n = row_n + 1
    row_n = row
    for ob in df['DOI Difference']:
        ws.cell(row=row_n, column=3).value = ob
        row_n = row_n + 1   
    row_n = row
    for ob in df['Early Adopter']:
        ws.cell(row=row_n, column=4).value = ob
        row_n = row_n + 1 
    row_n = row
    for ob in df['Late Adopter']:
        ws.cell(row=row_n, column=5).value = ob
        row_n = row_n + 1 
wb.save(file)